## Code for displaying table on Zeppelin

In [ ]:
import pandas as pd
# Reading csv file
df2 = pd.read_csv('death-rates-from-ambient-particulate-air-pollution.csv')
#Cleaning of DeathRate file and shortlist to the 8 countries
df2 = df2.rename(columns = {'Deaths - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized (Rate) (deaths per 100,000)':'Death rate (deaths per 100,000)'}).rename(columns = {'Entity':'Country'})
df2 = df2[df2['Country'].isin(['Niger','Qatar','Kuwait','Saudi Arabia', 'Egypt','China','United States','India'])]
df_16 = df2[df2['Year'].isin(['2016'])]
df16 = df_16.drop(columns = 'Code')

#Cleaning of Exposure file 
df = pd.read_csv('PM25-air-pollution.csv')
df = df.rename(columns = {'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) (micrograms per cubic meter)':'Annual mean exposure (µg/m³)'}).rename(columns = {'Entity':'Country'}).drop(columns = 'Code')
df = df[df['Year'].isin(['2016'])]
df = df.groupby('Country').mean().sort_values(by = ['Annual mean exposure (µg/m³)'], ascending=False) 
df['Country'] = df.index
df.index = range(len(df.index))
dfdf = df.drop(columns='Year')

#Merging the Exposure file to DeathRate file 
df_join = df16.merge(dfdf,how='right',on='Country')

#Cleaning the DALYS file
death = pd.read_csv('dalys-particulate-matter.csv')

death_1 = death.rename(columns = {'DALYs (Disability-Adjusted Life Years) - Ambient particulate matter pollution - Sex: Both - Age: Age-standardized (Rate) (DALYs per 100,000 individuals)':'Disability-Adjusted Life Years per 100,1000 individuals'}).rename(columns = {'Entity':'Country'}).drop(columns = 'Code')
death_1 = death_1[death_1['Year'].isin(['2016'])] 
death_1 = death_1[death_1['Country'].isin(['Niger','Qatar','Kuwait','Saudi Arabia', 'Egypt','China','United States','India'])]
death_2 = death_1.drop(columns = 'Year')

#Merging the DALYS file to the Main file
frame1 = [death_1,df_join]
full_df = df_join.merge(death_2,how='right',on='Country')

#Rearranging columns
cols = ['Country', 'Year', 'Annual mean exposure (µg/m³)','Death rate (deaths per 100,000)', 'Disability-Adjusted Life Years per 100,1000 individuals']
full_df = full_df[cols]

#Adding Population column
popul = pd.read_csv('API_SPPOPTOTL_DS2_en_csv_v2_10473719.csv')

hello = popul[popul['Country Name'].isin(['Qatar', 'Niger', 'Kuwait', 'Egypt', 'Saudi Arabia', 'China', 'India', 'United States','Egypt'])]
population_2016 = hello.drop(columns = ['Indicator Name', 'Country Code', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2017', '2018', 'Unnamed: 63'])
final_population_2016 = population_2016.rename(columns = {'2016' : 'population'})

ls = final_population_2016['population']
popls = ls.tolist()
popls.insert(1,95000000)
full_df['Population']= popls

#Converting DALYS column to days lost per individual
full_df['Disability-Adjusted Life Years per 100,1000 individuals']=full_df['Disability-Adjusted Life Years per 100,1000 individuals']*365/100000
full_df1 = full_df.rename(columns = {'Disability-Adjusted Life Years per 100,1000 individuals' : 'Disability-Adjusted Life Days per individual'})
full_df1


